# Part 5. DEAP Dataset + Common Spatial Pattern + SVM

In this part 5, we will focus on performing feature engineering using common spatial pattern analysis.  Common spatial pattern is about separating signals into additive composed components that has the maximium differences in variances.

Common spatial pattern is a very common dimensionality reduction techniques done on EEG signals.  The main difference between CSP and LDA is that CSP does not look at mean.  On the other hand, CSP is extremely similar to PCA (Principle Component Analysis) because it also employs eigenvalue decompositions but the slight difference is that CSP is done on two different signal windows thus you can say that CSP is a PCA made specificially for signals.

In this part, we shall extract common spatial patterns as features.  Lastly, let's try input the features into SVM and see if these features are useful for predicting the four valence-arousal classes that we have obtained from Part 1.

[Youtube - Lecture 7.3 Common Spatial Patterns](https://www.youtube.com/watch?v=zsOULC16USU)

## 1. Loading dataset

By the time I reach this tutorial, the class Dataset is gone. Therefore, I will load the data from my cache.

In [1]:
import numpy as np
import pickle

# Here I can load the data from cache (work a bit faster than loop through the /data/*.dat)
def save(data,filename):
    with open(f'cache/{filename}.pickle', 'wb') as handle:
        pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL)

def load(filename):
    with open(f'cache/{filename}.pickle', 'rb') as handle:
        data = pickle.load(handle)
    return data

data = load('data_valence')
label = load('label_valence')

print("Data shape: " , data.shape)  #15360 = 32 * 40 trials * 12 segments, 32 EEG channels, 672 samples
print("Label shape: ", label.shape)  #two classes of valence

Data shape:  (15360, 32, 672)
Label shape:  (15360,)


## 2. Common Spatial Pattern

For this technique, we are not going to code from scratch. Hooray!!!

The `mne` library already provide us a function to use.

- [Decoding in time-frequency space using Common Spatial Patterns (CSP)](https://mne.tools/stable/auto_examples/decoding/decoding_csp_timefreq.html)
- [Motor imagery decoding from EEG data using the Common Spatial Pattern (CSP)](https://mne.tools/stable/auto_examples/decoding/decoding_csp_eeg.html)

The API

- [mne.decoding.CSP](https://mne.tools/stable/generated/mne.decoding.CSP.html)

In quick, `mne.decoding.CSP` is a class and its usage is like sklearn class. (The `<obj>.fit_transform(x,y)`)

The key different between CSP and PCA is that the PCA only maximized the variance while CSP maximized the variance **based on label**

In the argument of `fit_transform`, `x` is your SVM ready data (n_samples, n_features) and `y` is the label (n_samples,)

Let's code

In [3]:
import mne
csp = mne.decoding.CSP(n_components=4, transform_into='average_power')

csp_data = csp.fit_transform(data.astype(np.float64),label)

print(type(csp_data), csp_data.shape)

Computing rank from data with rank=None
    Using tolerance 1.9e+03 (2.2e-16 eps * 32 dim * 2.6e+17  max singular value)
    Estimated rank (mag): 32
    MAG: rank 32 computed from 32 data channels with 0 projectors
Reducing data rank from 32 -> 32
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 1.7e+03 (2.2e-16 eps * 32 dim * 2.4e+17  max singular value)
    Estimated rank (mag): 32
    MAG: rank 32 computed from 32 data channels with 0 projectors
Reducing data rank from 32 -> 32
Estimating covariance using EMPIRICAL
Done.


LinAlgError: The leading minor of order 32 of B is not positive definite. The factorization of B could not be completed and no eigenvalues or eigenvectors were computed.

I have try various guess and I could found the way to fix this error. Therefore, I will aim toward what does the libry offer to you.

The code will run if only use [:3650]

Another knowledge I gain is the `mne.Epochs` convert data into `np.float64`.

In [4]:
csp = mne.decoding.CSP(n_components=4, transform_into='average_power')
csp_data = csp.fit_transform(data.astype(np.float64)[:3650],label[:3650])
print(type(csp_data), csp_data.shape)

Computing rank from data with rank=None
    Using tolerance 1e+03 (2.2e-16 eps * 32 dim * 1.5e+17  max singular value)
    Estimated rank (mag): 32
    MAG: rank 32 computed from 32 data channels with 0 projectors
Reducing data rank from 32 -> 32
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 8.7e+02 (2.2e-16 eps * 32 dim * 1.2e+17  max singular value)
    Estimated rank (mag): 32
    MAG: rank 32 computed from 32 data channels with 0 projectors
Reducing data rank from 32 -> 32
Estimating covariance using EMPIRICAL
Done.
<class 'numpy.ndarray'> (3650, 4)


The result is another `numpy array` with shape (n_samples, n_components). The configuration above is actually the default parameters of `mnd.decoding.CSP`.

The reason why we only have a single value of each component is that `transform_into` is set to `average_power`.

In [5]:
csp = mne.decoding.CSP(n_components=4, transform_into='csp_space')
csp_data = csp.fit_transform(data.astype(np.float64)[:3650],label[:3650])
print(type(csp_data), csp_data.shape)

Computing rank from data with rank=None
    Using tolerance 1e+03 (2.2e-16 eps * 32 dim * 1.5e+17  max singular value)
    Estimated rank (mag): 32
    MAG: rank 32 computed from 32 data channels with 0 projectors
Reducing data rank from 32 -> 32
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 8.7e+02 (2.2e-16 eps * 32 dim * 1.2e+17  max singular value)
    Estimated rank (mag): 32
    MAG: rank 32 computed from 32 data channels with 0 projectors
Reducing data rank from 32 -> 32
Estimating covariance using EMPIRICAL
Done.
<class 'numpy.ndarray'> (3650, 4, 672)


Now, that is more like it. The result is the same as performing PCA.

However, CSP component (`n_components`) can only be determine using cross-validation (documented in the API).

This means you can extract as many components as you like but it won't exceed the number of channels you have in the beginning.

In [7]:
csp = mne.decoding.CSP(n_components=100, transform_into='csp_space')
csp_data = csp.fit_transform(data.astype(np.float64)[:3650],label[:3650])
print(type(csp_data), csp_data.shape)

Computing rank from data with rank=None
    Using tolerance 1e+03 (2.2e-16 eps * 32 dim * 1.5e+17  max singular value)
    Estimated rank (mag): 32
    MAG: rank 32 computed from 32 data channels with 0 projectors
Reducing data rank from 32 -> 32
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 8.7e+02 (2.2e-16 eps * 32 dim * 1.2e+17  max singular value)
    Estimated rank (mag): 32
    MAG: rank 32 computed from 32 data channels with 0 projectors
Reducing data rank from 32 -> 32
Estimating covariance using EMPIRICAL
Done.
<class 'numpy.ndarray'> (3650, 32, 672)


the data we obtained is a time-series data. Thus mean the followed task is spectral analysis. (well, it does not make sense to do asymmetry and connectivity since the data is no longer a brain region space anymore).


## CSP with another Library

https://github.com/spolsley/common-spatial-patterns

In [2]:
# https://github.com/spolsley/common-spatial-patterns

# Common Spatial Pattern implementation in Python, used to build spatial filters for identifying task-related activity.
import numpy as np
import scipy.linalg as la

# CSP takes any number of arguments, but each argument must be a collection of trials associated with a task
# That is, for N tasks, N arrays are passed to CSP each with dimensionality (# of trials of task N) x (feature vector)
# Trials may be of any dimension, provided that each trial for each task has the same dimensionality,
# otherwise there can be no spatial filtering since the trials cannot be compared
def CSP(*tasks):
	if len(tasks) < 2:
		print("Must have at least 2 tasks for filtering.")
		return (None,) * len(tasks)
	else:
		filters = ()
		# CSP algorithm
		# For each task x, find the mean variances Rx and not_Rx, which will be used to compute spatial filter SFx
		iterator = range(0,len(tasks))
		for x in iterator:
			# Find Rx
			Rx = covarianceMatrix(tasks[x][0])
			for t in range(1,len(tasks[x])):
				Rx += covarianceMatrix(tasks[x][t])
			Rx = Rx / len(tasks[x])

			# Find not_Rx
			count = 0
			not_Rx = Rx * 0
			for not_x in [element for element in iterator if element != x]:
				for t in range(0,len(tasks[not_x])):
					not_Rx += covarianceMatrix(tasks[not_x][t])
					count += 1
			not_Rx = not_Rx / count

			# Find the spatial filter SFx
			SFx = spatialFilter(Rx,not_Rx)
			filters += (SFx,)

			# Special case: only two tasks, no need to compute any more mean variances
			if len(tasks) == 2:
				filters += (spatialFilter(not_Rx,Rx),)
				break
		return filters

# covarianceMatrix takes a matrix A and returns the covariance matrix, scaled by the variance
def covarianceMatrix(A):
	Ca = np.dot(A,np.transpose(A))/np.trace(np.dot(A,np.transpose(A)))
	return Ca

# spatialFilter returns the spatial filter SFa for mean covariance matrices Ra and Rb
def spatialFilter(Ra,Rb):
	R = Ra + Rb
	E,U = la.eig(R)

	# CSP requires the eigenvalues E and eigenvector U be sorted in descending order
	ord = np.argsort(E)
	ord = ord[::-1] # argsort gives ascending order, flip to get descending
	E = E[ord]
	U = U[:,ord]

	# Find the whitening transformation matrix
	P = np.dot(np.sqrt(la.inv(np.diag(E))),np.transpose(U))

	# The mean covariance matrices may now be transformed
	Sa = np.dot(P,np.dot(Ra,np.transpose(P)))
	Sb = np.dot(P,np.dot(Rb,np.transpose(P)))

	# Find and sort the generalized eigenvalues and eigenvector
	E1,U1 = la.eig(Sa,Sb)
	ord1 = np.argsort(E1)
	ord1 = ord1[::-1]
	E1 = E1[ord1]
	U1 = U1[:,ord1]

	# The projection matrix (the spatial filter) may now be obtained
	SFa = np.dot(np.transpose(U1),P)
	return SFa.astype(np.float32)

In [3]:
filters = CSP(data[label==0], data[label==1])
print(type(filters), filters[0].shape, filters[1].shape)

<class 'tuple'> (32, 32) (32, 32)


/tmp/ipykernel_17619/3811170600.py:78: ComplexWarning: Casting complex values to real discards the imaginary part
  return SFa.astype(np.float32)


Ah ha, I have not yet given up. I search for other library and here we are continue to code. Sorry fork. You stuck with me for a bit longer.

As you can see, this library calculate the filter base on the given data. We will you the filter to convert from `brain region space` to `csp-space`.

The filter will need to be used base on the label.

In [28]:
(data[0].T @ filters[label[0].astype(int)]).T.shape

(32, 672)

In [4]:
from tqdm.notebook import tqdm
csp_data = []
for index in tqdm(range(len(data))):
    
    # (672, 32) @ (32, 32) = (672, 32)
    csp = data[index].T @ filters[label[index].astype(int)]
    # (1, 32, 672)
    csp = np.expand_dims(csp.T, axis=0)
    csp_data.append(csp)

csp_data = np.vstack(csp_data)
print(csp_data.shape)

  0%|          | 0/15360 [00:00<?, ?it/s]

(15360, 32, 672)


In [5]:
from mne_features.feature_extraction import FeatureExtractor
import time
start = time.time()

bands = [(0,4), (4,8), (8,12), (12,30), (30,64)]
# [alias_feature_function]__[optional_param]
params = dict({
    'pow_freq_bands__log':True,
    'pow_freq_bands__normalize':False,
    'pow_freq_bands__freq_bands':bands
})
fe = FeatureExtractor(sfreq=128, selected_funcs=['pow_freq_bands'],params=params,n_jobs=8)
X = fe.fit_transform(X=csp_data)
print(X.shape, time.time() - start)

/root/projects/.venv/lib/python3.8/site-packages/mne_features/mock_numba.py:12: UserWarning: Numba needs NumPy 1.21 or less. Your code will be slower.
  warn('{}. Your code will be slower.'.format(err))
/root/projects/.venv/lib/python3.8/site-packages/mne_features/mock_numba.py:12: UserWarning: Numba needs NumPy 1.21 or less. Your code will be slower.
  warn('{}. Your code will be slower.'.format(err))
/root/projects/.venv/lib/python3.8/site-packages/mne_features/mock_numba.py:12: UserWarning: Numba needs NumPy 1.21 or less. Your code will be slower.
  warn('{}. Your code will be slower.'.format(err))
/root/projects/.venv/lib/python3.8/site-packages/mne_features/mock_numba.py:12: UserWarning: Numba needs NumPy 1.21 or less. Your code will be slower.
  warn('{}. Your code will be slower.'.format(err))
/root/projects/.venv/lib/python3.8/site-packages/mne_features/mock_numba.py:12: UserWarning: Numba needs NumPy 1.21 or less. Your code will be slower.
  warn('{}. Your code will be slower.

(15360, 160) 66.80781841278076


In [41]:
def train_model(X_ori,y_ori, kernel='rbf'):
    # Make a copy because I am paranoid
    X,y = X_ori.copy(), y_ori.copy()

    from sklearn.svm import SVC
    from sklearn.utils import shuffle
    from sklearn.model_selection import cross_val_score

    # 
    X_shuff,y_shuff = shuffle(X,y)
    model = SVC(kernel=kernel,max_iter=10000)
    cross = cross_val_score(model, X_shuff, y_shuff, cv=3)

    model = SVC(kernel=kernel, max_iter=10000)
    model.fit(X_shuff, y_shuff)
    ans = model.predict(X_shuff)
    acc = sum(ans == y_shuff) / len(y_shuff)
    return model, acc, cross

for kernel in ['linear','poly','rbf', 'sigmoid']:
    start = time.time()
    model, acc, cross = train_model(X, label, kernel=kernel)
    # We can save the model and reuse it later
    print(f"\tKernel={kernel}| Acc={round(acc,5)} | 3-CV score={round(cross.mean(),5)} STD={round(cross.std(),5)}| Time spend={time.time() - start}")

/root/projects/.venv/lib/python3.8/site-packages/sklearn/svm/_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/root/projects/.venv/lib/python3.8/site-packages/sklearn/svm/_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/root/projects/.venv/lib/python3.8/site-packages/sklearn/svm/_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/root/projects/.venv/lib/python3.8/site-packages/sklearn/svm/_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


	Kernel=linear| Acc=0.64616 | 3-CV score=0.70352 STD=0.03324| Time spend=73.69449186325073


/root/projects/.venv/lib/python3.8/site-packages/sklearn/svm/_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/root/projects/.venv/lib/python3.8/site-packages/sklearn/svm/_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/root/projects/.venv/lib/python3.8/site-packages/sklearn/svm/_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/root/projects/.venv/lib/python3.8/site-packages/sklearn/svm/_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


	Kernel=poly| Acc=0.45228 | 3-CV score=0.56914 STD=0.11847| Time spend=7.272881507873535
	Kernel=rbf| Acc=0.67826 | 3-CV score=0.67357 STD=0.00033| Time spend=139.15341567993164
	Kernel=sigmoid| Acc=0.55312 | 3-CV score=0.55312 STD=0.0| Time spend=105.43437027931213


Okay, the result is not very pleasing. However, we did use all 32 csp components. As suggested by the mne folk, we should play around with the number of components

In [47]:
csp_data[:,:4,:].shape

(15360, 4, 672)

In [7]:
for n_component in range(1,32+1):
    print(f'{n_component=}')
    bands = [(0,4), (4,8), (8,12), (12,30), (30,64)]
    # [alias_feature_function]__[optional_param]
    params = dict({
        'pow_freq_bands__log':True,
        'pow_freq_bands__normalize':False,
        'pow_freq_bands__freq_bands':bands
    })
    fe = FeatureExtractor(sfreq=128, selected_funcs=['pow_freq_bands'],params=params,n_jobs=8)

    ##### Here I select component #####
    X = fe.fit_transform(X=csp_data[:,:n_component,:])
    print(X.shape, time.time() - start)

    start = time.time()
    model, acc, cross = train_model(X, label, kernel='rbf')
    # We can save the model and reuse it later
    print(f"\tKernel='rbf'| Acc={round(acc,5)} | 3-CV score={round(cross.mean(),5)} STD={round(cross.std(),5)}| Time spend={time.time() - start}")
    print('----'*5)

n_component=1
(15360, 5) 82.39137411117554
	Kernel='rbf'| Acc=0.67435 | 3-CV score=0.67188 STD=0.00295| Time spend=30.879997491836548
--------------------
n_component=2
(15360, 10) 38.85746693611145
	Kernel='rbf'| Acc=0.6748 | 3-CV score=0.67109 STD=0.00804| Time spend=33.660973072052
--------------------
n_component=3
(15360, 15) 40.304635524749756
	Kernel='rbf'| Acc=0.6748 | 3-CV score=0.67285 STD=0.00732| Time spend=38.41354560852051
--------------------
n_component=4
(15360, 20) 44.97115874290466
	Kernel='rbf'| Acc=0.67454 | 3-CV score=0.67161 STD=0.00369| Time spend=37.92103624343872
--------------------
n_component=5
(15360, 25) 44.849576234817505
	Kernel='rbf'| Acc=0.67786 | 3-CV score=0.67409 STD=0.00113| Time spend=43.60341739654541
--------------------
n_component=6
(15360, 30) 50.73816728591919
	Kernel='rbf'| Acc=0.67754 | 3-CV score=0.67285 STD=0.0036| Time spend=50.42497944831848
--------------------
n_component=7
(15360, 35) 57.738218784332275
	Kernel='rbf'| Acc=0.67799 |

After the comprehensive run, I am a bit disappointed that I could not get the result higher than 67% accuracy. Below I re-run the spectral analysis from the `brain region space` data and confirm that it achieves a higher accuracy.

In [10]:
bands = [(0,4), (4,8), (8,12), (12,30), (30,64)]
# [alias_feature_function]__[optional_param]
params = dict({
    'pow_freq_bands__log':True,
    'pow_freq_bands__normalize':False,
    'pow_freq_bands__freq_bands':bands
})
fe = FeatureExtractor(sfreq=128, selected_funcs=['pow_freq_bands'],params=params,n_jobs=8)

##### Here I select component #####
X = fe.fit_transform(X=data)
print(X.shape, time.time() - start)

start = time.time()
model, acc, cross = train_model(X, label, kernel='rbf')
# We can save the model and reuse it later
print(f"\tKernel='rbf'| Acc={round(acc,5)} | 3-CV score={round(cross.mean(),5)} STD={round(cross.std(),5)}| Time spend={time.time() - start}")
print('----'*5)

(15360, 160) 209.67438411712646
	Kernel='rbf'| Acc=0.70436 | 3-CV score=0.67344 STD=0.00361| Time spend=189.62864065170288
--------------------
